* 전체 N개 작업을 N/P개씩 P개 작업 그룹으로 나눈다.
* 각 스레드 또는 코어는 작업 그룹을 하나씩 할당받는다.
* 각 스레드는 할당받은 영역의 값을 계산하고, 그 합을 대표 스레드에게 전달한다.
* 모든 스레드는 각 작업을 마칠 때까지 기다린다.
* 모든 코어가 계산한 합을 대표 스레드가 취합한 후, 그 값을 하나로 더한다.

이 간단한 병렬 처리 알고리즘을 의사 코드(psedu code)로 표현하면 다음과 같은 형태가 된다.
1~2 단계에서 각 코어가 담당할 작업 구역이 my_first와 my_end로 결정되고, 모든 스레드가 ComputeMySum 함수를 통해 자신에게 할당된 작업 그룹을 처리한다.
다음으로 대표 스레드가 아닌 스레드들은 자신의 계산 결과를 대표 스레드에 전달한다.
대표ㅛ 스레드는 모든 스레드의 결과를 취합해서 최종 결과를 계산한다.

컴파일러가 제공하는 자동 병렬화 기능이나 병렬 처리 코드 생성 도구 등을 사용하면 이와 같은 간단한 병렬화를 제공해줄 것이다.
그렇다면 이러한 병렬 처리 알고리즘을 통해 어느 정도 성능이 향상될까?
계산 편의상, 값을 구해서 더하는 연산, 값을 전달받고 더하는 연산 모두 1초가 걸린다고 가정해보자.
만약 데이터 수가 24, 프로세스 수가 8이라면 각 스레드가 3(24 / 8) 개의 값을 구해서 더하고, 대표 스레드는 자신을 제외한 7명에게 데이터를 전달받고 합산해야 한다.

그 결과, 각 스레드의 값 계산 3초와 취합에 소요된 7초를 더해서 총 10초의 시간이 소요된다.
직렬 처리 시 하나의 코어가 24개의 값을 계산하고 더하기 때문에 24초가 걸린다.
즉, 병렬 처리를 통해 2.4 배의 성능 향상을 얻는다.
성능이 향상딘다는 점에서는 긍정적이지만 8개의 코어를 사용했는데 2.4배가 증가한 성능은 조금 아쉽다는 생각이 들 것이다.

더 높은 성능을 얻기 위해 사용하는 코어의 수를 늘리는 경우를 생각해보자.
각 스레드는 N / P 번의 값을 계산 및 합하고 대표 스레드는 (p-1) 번 취합을 수행하므로, N과 P에 따른 연산 소요 시간은 다음과 같이 계산할 수 있다.

* 연산 시간 = N / P + (p - 1)

P가 증가함에 따른 연산 시간의 결과는 어떻게 될까?
일정 수준까지는 성능이 향상되지만, 그 이후부터는 코어 수가 증가하는데도 오히려 성능은 떨어진다.
코어 수가 늘어나는 구간도, 비록 성능이 향상되기는 하지만 그 효율은 점점 낮아지는 것을 확인할 수 있다.

이는 코어의 수가 증가함에 따라 각각의 값을 구하고 작업 그룹 내 합을 구하는 시간은 감소하지만, 그 결과 값을 수집하기 위한 부하(overhead)가 너무 커져서 병렬 처리의 효율을 낮추기 때문에 발생하는 결과다.

그렇다면 더 효율적인 병렬 처리 알고리즘은 없을까?

현재 알고리즘을 보면, 대표 스레드가 전체 결과를 취합하는 부분이 큰 시간을 차지하는 것을 확인할 수 있다.
그리고 그 이유 중 하나는 대표 스레드가 혼자 일하기 때문이다.
그렇다면 최종 취합을 다른 스레드들이 도와줄 수 있는 방법이 없을까?

알고리즘을 다음과 같이 수정해보자.

1. 전체 N개 작업을 N/P 개씩 P개 작업 그룹으로 나눈다.
2. 각 스레드 또는 코어는 작업 그룹을 하나씩 할당받는다.
3. 각 스레드는 할당받은 영역의 값을 계산한다.
4. 모든 스레드가 각 작업을 마칠 때까지 기다린다.
5. (i + 2^(k-1)) 번 코어는 작업을 종료한다.(최초의 k=1)
6. i번 스레드는 (i+2^(k-1)) 번 코어의 결과 값을 가져와서 더한다.
7. k를 1 증가시킨다.
8. 작업에 참여하는 스레드가 더 이상 없을 때까지 5 ~ 7 단계를 반복한다.

N이 24, P가 8일 떄에 총 연산에 소요되는 시간을 계산해보면 취합의 각 단계에서 소요되는 시간이 1초이고 총 3 단께를 통해 전체를 취합할 수 있다.
따라서 각 스레드가 자신에게 할당된 그룹을 처리하는 시간 3초를 더해서 총 6초의 시간이 걸림을 알 수 있다.
이는 직렬 처리 대비 성능이 4배가 된다.
N과 P에 따라 개선된 병렬 처리 알고리즘이 소요하는 시간은 아래와 같이 계산할 수 있다.

개선된 알고리즘의 연산 시간 = N / P + log_2 N

개선된 병렬 알고리즘을 사용하면, 사용하는 코어의 수가 증가함에 따라 지속적으로 성능이 향상되는 것을 볼 수 있다.
즉, 병렬 처리 영역을 최대한 늘림으로써 (값 계산 + 취합 과정), 여러 개의 코어를 사용하는 효율을 극대화하는 것이다.

같은 수의 코어를 사용하더라도 알고리즘을 어떻게 설계하는지에 따라 그 성능이 크게 차이가 날 수 있다.
효율적인 알고리즘 설계를 위해서는 해결할는 문제의 특성 및 연산 자원에 대한 깊은 이해가 필요하다.
이러한 문제 및 자원에 대한 이해를 바탕으로 병렬 처리 알고리즘을 설계하고 개발하는 것은 현재의 자동 병렬화 기술들이 달성하기 힘든 과제다.
즉, 우리가 직접 설계하고 구현해야 하며, 이를 위해 GPU 프로그래밍과 같은 병렬 처리 프로그래밍을 학습해야 한다.